M5 Forecasting - Accuracy

Note: This is one of the two complementary competitions that together comprise the M5 forecasting challenge. Can you estimate, as precisely as possible, the point forecasts of the unit sales of various products sold in the USA by Walmart?

How much camping gear will one store sell each month in a year? To the uninitiated, calculating sales at this level may seem as difficult as predicting the weather. Both types of forecasting rely on science and historical data. While a wrong weather forecast may result in you carrying around an umbrella on a sunny day, inaccurate business forecasts could result in actual or opportunity losses. In this competition, in addition to traditional forecasting methods you’re also challenged to use machine learning to improve forecast accuracy.

The Makridakis Open Forecasting Center (MOFC) at the University of Nicosia conducts cutting-edge forecasting research and provides business forecast training. It helps companies achieve accurate predictions, estimate the levels of uncertainty, avoiding costly mistakes, and apply best forecasting practices. The MOFC is well known for its Makridakis Competitions, the first of which ran in the 1980s.

In this competition, the fifth iteration, you will use hierarchical sales data from Walmart, the world’s largest company by revenue, to forecast daily sales for the next 28 days. The data, covers stores in three US States (California, Texas, and Wisconsin) and includes item level, department, product categories, and store details. In addition, it has explanatory variables such as price, promotions, day of the week, and special events. Together, this robust dataset can be used to improve forecasting accuracy.

If successful, your work will continue to advance the theory and practice of forecasting. The methods used can be applied in various business areas, such as setting up appropriate inventory or service levels. Through its business support and training, the MOFC will help distribute the tools and knowledge so others can achieve more accurate and better calibrated forecasts, reduce waste and be able to appreciate uncertainty and its risk implications.

Evaluation:
This competition uses a Weighted Root Mean Squared Scaled Error (RMSSE). Extensive details about the metric, scaling, and weighting can be found in the [M5 Participants Guide](https://mofc.unic.ac.cy/m5-competition/).

It is important to notice that this notebook must be seen together with the notebooks "M5 Forecasting - Accuracy Data study.ipynb" and "M5 Forecasting - Accuracy  - Variables". All the necessary data verification, and the construction of the variables are done in those two notebook. This notebook has the final lightgbm used in this competition.

This notebook is similar to "M5 Forecasting - Accuracy - Model v2". However, we wiil not use the lagged demand.

In [1]:
import gc
import time
import pandas as pd
import numpy as np
import re
import seaborn as sns

import lightgbm as lgb

In [2]:
path = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 3\\data_by_id\\'
path_holdout = 'C:\\Users\\maxwi\\Python\\Kaggle\\M5 Forecasting - Accuracy\\Modelo 3\\holdout_by_id\\'

In [3]:
#Root Mean Squared Scaled Error (RMSSE)
def RMSSE(train, actual, forecast, h = 28):
    sum_1 = 0
    for i in range(len(actual)):
        sum_1 += (actual.iloc[i] - forecast[i])**2
        
    sum_2 = 0
    for i in range(1, len(train)):
        sum_2 += (train.iloc[i] - train.iloc[i - 1])**2
        
    rmsse = round(np.sqrt((1/h)*(sum_1/sum_2)), 4)
    return rmsse

In [4]:
sales_train_validation = pd.read_csv('sales_train_evaluation.csv') #Only used to take the products ids.
products_ids = sales_train_validation['id'].unique()
products_ids_size = len(products_ids)
del sales_train_validation

In [1]:
#Define variables. We have a total of 178 variables.

#Continuous variables. We have 14 continuous variables.
continous_variables = ['month', 'wday',
    'demand_rolling_mean_t28_7', 'demand_rolling_mean_t28_30', 'demand_rolling_mean_t28_90', 'demand_rolling_mean_t28_180',
    'demand_rolling_mean_t28_365', 'demand_rolling_std_t28_7', 'demand_rolling_std_t28_30', 'demand_rolling_std_t28_90', 
    'demand_rolling_std_t28_180', 'demand_rolling_std_t28_365', 'demand_rolling_skew_t28_30','demand_rolling_kurt_t28_30']

#Categorical variables. We have 164 categoric variables.
categoric_variables = ['season', 'month_fase', 'week_fase', 'SuperBowl',
     'ValentinesDay', 'PresidentsDay', 'LentStart', 'LentWeek2',
     'StPatricksDay', 'Purim End', 'OrthodoxEaster', 'Pesach End',
     'Cinco De Mayo', "Mother's day", 'MemorialDay', 'NBAFinalsStart',
     'NBAFinalsEnd', "Father's day", 'IndependenceDay', 'Ramadan starts',
     'Eid al-Fitr', 'LaborDay', 'ColumbusDay', 'Halloween',
     'EidAlAdha', 'VeteransDay', 'Thanksgiving', 'Christmas',
     'Chanukah End', 'NewYear', 'OrthodoxChristmas', 'MartinLutherKingDay',
     'Easter', 'Sporting', 'Cultural', 'National',
     'Religious', 'snap', 'snap_other', 'snap_only_other',
     'SuperBowl_near', 'SuperBowl_week', 'SuperBowl_weekend', 'ValentinesDay_near',
     'ValentinesDay_week', 'ValentinesDay_weekend', 'PresidentsDay_near', 'PresidentsDay_week',
     'PresidentsDay_weekend', 'LentStart_near', 'LentStart_week', 'LentStart_weekend',
     'LentWeek2_near', 'LentWeek2_week', 'LentWeek2_weekend', 'StPatricksDay_near',
     'StPatricksDay_week', 'StPatricksDay_weekend', 'Purim End_near', 'Purim End_week',
     'Purim End_weekend', 'OrthodoxEaster_near', 'OrthodoxEaster_week', 'OrthodoxEaster_weekend',
     'Pesach End_near', 'Pesach End_week', 'Pesach End_weekend', 'Cinco De Mayo_near',
     'Cinco De Mayo_week', 'Cinco De Mayo_weekend', "Mother's day_near", "Mother's day_week",
     "Mother's day_weekend", 'MemorialDay_near', 'MemorialDay_week', 'MemorialDay_weekend',
     'NBAFinalsStart_near', 'NBAFinalsStart_week', 'NBAFinalsStart_weekend', 'NBAFinalsEnd_near',
     'NBAFinalsEnd_week', 'NBAFinalsEnd_weekend', "Father's day_near", "Father's day_week",
     "Father's day_weekend", 'IndependenceDay_near', 'IndependenceDay_week', 'IndependenceDay_weekend',
     'Ramadan starts_near', 'Ramadan starts_week', 'Ramadan starts_weekend', 'Eid al-Fitr_near',
     'Eid al-Fitr_week', 'Eid al-Fitr_weekend', 'LaborDay_near', 'LaborDay_week',
     'LaborDay_weekend', 'ColumbusDay_near', 'ColumbusDay_week', 'ColumbusDay_weekend',
     'Halloween_near', 'Halloween_week', 'Halloween_weekend', 'EidAlAdha_near',
     'EidAlAdha_week', 'EidAlAdha_weekend', 'VeteransDay_near', 'VeteransDay_week',
     'VeteransDay_weekend', 'Thanksgiving_near', 'Thanksgiving_week', 'Thanksgiving_weekend',
     'Christmas_near', 'Christmas_week', 'Christmas_weekend', 'Chanukah End_near',
     'Chanukah End_week', 'Chanukah End_weekend', 'NewYear_near', 'NewYear_week',
     'NewYear_weekend', 'OrthodoxChristmas_near', 'OrthodoxChristmas_week', 'OrthodoxChristmas_weekend',
     'MartinLutherKingDay_near', 'MartinLutherKingDay_week', 'MartinLutherKingDay_weekend', 'Easter_near',
     'Easter_week', 'Easter_weekend', 'Sporting_near', 'Sporting_week',
     'Sporting_weekend', 'Cultural_near', 'Cultural_week', 'Cultural_weekend',
     'National_near', 'National_week', 'National_weekend', 'Religious_near',
     'Religious_week', 'Religious_weekend', 'price_change_t1', 'price_change_t1_bin',
     'price_change_max_t4', 'price_change_max_t13', 'price_change_max_t26', 'price_change_max_t52',
     'price_change_max_t4_bin', 'price_change_max_t13_bin', 'price_change_max_t26_bin', 'price_change_max_t52_bin',
     'rolling_price_std_t4', 'rolling_price_std_t13', 'rolling_price_std_t26', 'rolling_price_std_t52',
     'price_change_mean_t4', 'price_change_mean_t13', 'price_change_mean_t26', 'price_change_mean_t52',
     'price_change_mean_t4_bin', 'price_change_mean_t13_bin', 'price_change_mean_t26_bin', 'price_change_mean_t52_bin']

variables_all = []
for e in continous_variables:
    variables_all.append(e)
for e in categoric_variables:
    variables_all.append(e)

['month',
 'wday',
 'demand_rolling_mean_t28_7',
 'demand_rolling_mean_t28_30',
 'demand_rolling_mean_t28_90',
 'demand_rolling_mean_t28_180',
 'demand_rolling_mean_t28_365',
 'demand_rolling_std_t28_7',
 'demand_rolling_std_t28_30',
 'demand_rolling_std_t28_90',
 'demand_rolling_std_t28_180',
 'demand_rolling_std_t28_365',
 'demand_rolling_skew_t28_30',
 'demand_rolling_kurt_t28_30',
 'season',
 'month_fase',
 'week_fase',
 'SuperBowl',
 'ValentinesDay',
 'PresidentsDay',
 'LentStart',
 'LentWeek2',
 'StPatricksDay',
 'Purim End',
 'OrthodoxEaster',
 'Pesach End',
 'Cinco De Mayo',
 "Mother's day",
 'MemorialDay',
 'NBAFinalsStart',
 'NBAFinalsEnd',
 "Father's day",
 'IndependenceDay',
 'Ramadan starts',
 'Eid al-Fitr',
 'LaborDay',
 'ColumbusDay',
 'Halloween',
 'EidAlAdha',
 'VeteransDay',
 'Thanksgiving',
 'Christmas',
 'Chanukah End',
 'NewYear',
 'OrthodoxChristmas',
 'MartinLutherKingDay',
 'Easter',
 'Sporting',
 'Cultural',
 'National',
 'Religious',
 'snap',
 'snap_other',
 '

In [15]:
stoped_at = 'HOBBIES_1_001_CA_1_evaluation'  #  Used to continue the code from the last product saved. For safety, we will run the last product saved again.
last_product = 'FOODS_3_827_WI_3_evaluation'
stoped_at_index = np.where(products_ids == stoped_at)[0][0]
last_product_index = np.where(products_ids == last_product)[0][0]
products_ids_size =  last_product_index - stoped_at_index + 1

progress = 0  #Usefull to see the progress of the code. 
progress_1000 = 1
start = time.time()

#for product in products_ids[stoped_at_index : last_product_index + 1]:
for product in products_ids[stoped_at_index : ]:
    data_variables = pd.read_hdf(path + product + '.h5')
    data_variables.reset_index(inplace = True)
    
    #Set type of variables as categoric
    for e in categoric_variables:
        data_variables[e] = data_variables[e].astype('category')
        
    #Set parameter for the Lightgbm model
    observations = len(data_variables)
    lgb_params = {
        "objective": "poisson",
        "metric": "rmse",
        "force_row_wise": True,
        "bagging_freq": 1,
        "learning_rate": 0.01,
        "lambda_l2": 0.1,
        'verbosity': 1,
        'num_iterations': 2000,
        #'early_stopping_round': 200,
        'num_leaves': 220,
        "min_data_in_leaf": int(observations*0.10),
        'seed': 1
    }
    
    train = data_variables[0:-56][variables_all].copy()
    test = data_variables[-56:][variables_all].copy()
    test.reset_index(inplace = True)
    test.drop('index', axis = 1, inplace = True)
    
    train_demand = data_variables[0:-56]['demand'].copy()
    demand_aux = train_demand.copy()  #used to recalculate some variables based on demand
    
    lgb_train = lgb.Dataset(data = train, label = train_demand, feature_name = variables_all)
    
    model = lgb.train(train_set = lgb_train, params = lgb_params)
    
    #Forecast
    predicted_demand = []
    for i in range(0, 56):
        predicted_demand_aux = model.predict(test[i : i + 1])
        predicted_demand.append(predicted_demand_aux[0])
        demand_aux = demand_aux.append(pd.Series(predicted_demand_aux[0]))
        

            
    #Make holdout
    holdout_1 = {} #Forecast to the public leaderboard
    holdout_1['id'] = f"{product[0:-10]}validation"
    for i in range(28):
        holdout_1['F' + str(i + 1)] = predicted_demand[i]
    holdout_1 = pd.DataFrame([holdout_1])
    
    holdout_2 = {} #Forecast to the private leaderboard
    holdout_2['id'] = product
    for i in range(28):
        holdout_2['F' + str(i + 1)] = predicted_demand[i + 28]
    holdout_2 = pd.DataFrame([holdout_2])

    holdout = pd.concat([holdout_1, holdout_2], ignore_index = True)
    holdout.to_hdf(path_holdout + product + '.h5', key = product, mode = 'w')
    
    progress += 1
    if progress == progress_1000 * 1000:

        progress_per = round(progress / products_ids_size, 4)
        print(progress_per)
        progress_1000 +=1
        
        end = time.time()
        elapsed = int(round(end - start, 0))
        total_run_time =  int(round(elapsed / (progress_per), 0))
        time_to_finish = int(round(elapsed / (progress_per), 0)) - elapsed
        print('Elapsed: {:02d}:{:02d}:{:02d}'.format(elapsed // 3600, (elapsed % 3600 // 60), elapsed % 60))
        print('Total run time: {:02d}:{:02d}:{:02d}'.format(total_run_time // 3600, (total_run_time % 3600 // 60), total_run_time % 60))
        print('Time to finish: {:02d}:{:02d}:{:02d}'.format(time_to_finish // 3600, (time_to_finish % 3600 // 60), time_to_finish % 60))
        print()
        
print()
print("OK!")

0.0328
Elapsed: 03:05:11
Total run time: 94:05:50
Time to finish: 91:00:39

0.0656
Elapsed: 06:09:42
Total run time: 93:55:40
Time to finish: 87:45:58

0.0984
Elapsed: 09:15:25
Total run time: 94:04:29
Time to finish: 84:49:04

0.1312
Elapsed: 12:19:37
Total run time: 93:57:19
Time to finish: 81:37:42

0.164
Elapsed: 15:23:46
Total run time: 93:52:43
Time to finish: 78:28:57

0.1968
Elapsed: 18:27:09
Total run time: 93:45:46
Time to finish: 75:18:37

0.2296
Elapsed: 21:31:19
Total run time: 93:44:12
Time to finish: 72:12:53

0.2624
Elapsed: 24:36:29
Total run time: 93:46:51
Time to finish: 69:10:22

0.2952
Elapsed: 27:43:07
Total run time: 93:53:52
Time to finish: 66:10:45

0.328
Elapsed: 30:46:50
Total run time: 93:50:35
Time to finish: 63:03:45

0.3608
Elapsed: 33:49:24
Total run time: 93:44:43
Time to finish: 59:55:19



KeyboardInterrupt: 

In [16]:
#Define holdout
stoped_at = products_ids[0]  #  Used to continue the code from the last product saved. For safety, we will run the last product saved again.

holdout_accuracy = pd.read_hdf(path_holdout + stoped_at + '.h5')

progress = 0  #Usefull to see the progress of the code. 
progress_1000 = 1
start = time.time()

for product in products_ids[1:]:
    holdout_accuracy_aux = pd.read_hdf(path_holdout + product + '.h5')
    
    holdout_accuracy = pd.concat([holdout_accuracy, holdout_accuracy_aux], ignore_index=True)

    progress += 1
    if progress == progress_1000 * 1000:

        progress_per = round(progress / products_ids_size, 4)
        print(progress_per)
        progress_1000 +=1
        
        end = time.time()
        elapsed = int(round(end - start, 0))
        total_run_time =  int(round(elapsed / (progress_per), 0))
        time_to_finish = int(round(elapsed / (progress_per), 0)) - elapsed
        print('Elapsed: {:02d}:{:02d}:{:02d}'.format(elapsed // 3600, (elapsed % 3600 // 60), elapsed % 60))
        print('Total run time: {:02d}:{:02d}:{:02d}'.format(total_run_time // 3600, (total_run_time % 3600 // 60), total_run_time % 60))
        print('Time to finish: {:02d}:{:02d}:{:02d}'.format(time_to_finish // 3600, (time_to_finish % 3600 // 60), time_to_finish % 60))
        print()

holdout_accuracy.to_csv("holdout_accuracy.csv", index=False)

0.0328
Elapsed: 00:00:09
Total run time: 00:04:34
Time to finish: 00:04:25

0.0656
Elapsed: 00:00:18
Total run time: 00:04:34
Time to finish: 00:04:16

0.0984
Elapsed: 00:00:27
Total run time: 00:04:34
Time to finish: 00:04:07

0.1312
Elapsed: 00:00:37
Total run time: 00:04:42
Time to finish: 00:04:05

0.164
Elapsed: 00:00:47
Total run time: 00:04:47
Time to finish: 00:04:00

0.1968
Elapsed: 00:00:57
Total run time: 00:04:50
Time to finish: 00:03:53

0.2296
Elapsed: 00:01:08
Total run time: 00:04:56
Time to finish: 00:03:48

0.2624
Elapsed: 00:01:19
Total run time: 00:05:01
Time to finish: 00:03:42

0.2952
Elapsed: 00:01:31
Total run time: 00:05:08
Time to finish: 00:03:37

0.328
Elapsed: 00:01:42
Total run time: 00:05:11
Time to finish: 00:03:29

0.3608
Elapsed: 00:01:51
Total run time: 00:05:08
Time to finish: 00:03:17

0.3936
Elapsed: 00:02:00
Total run time: 00:05:05
Time to finish: 00:03:05

0.4264
Elapsed: 00:02:09
Total run time: 00:05:03
Time to finish: 00:02:54

0.4592
Elapsed

In [17]:
holdout_accuracy

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.714343,0.733201,0.797134,0.912670,0.912175,1.145796,0.830001,0.873325,0.970058,...,0.956718,1.017517,0.802935,0.831025,0.853748,0.879291,0.819820,0.750022,1.162521,0.908328
1,HOBBIES_1_001_CA_1_evaluation,0.931565,0.842672,0.731990,0.860550,0.860084,1.103890,0.818146,0.799563,0.731815,...,0.816135,0.963181,0.969129,0.940354,0.940354,0.859457,1.037727,1.096337,1.338920,1.030632
2,HOBBIES_1_002_CA_1_validation,0.113675,0.116560,0.126434,0.133826,0.133826,0.180949,0.347894,0.321495,0.302967,...,0.305529,0.428628,0.308804,0.289620,0.291470,0.336835,0.345131,0.350064,0.429807,0.174232
3,HOBBIES_1_002_CA_1_evaluation,0.204301,0.195724,0.204003,0.201817,0.297835,0.331508,0.319895,0.325301,0.176902,...,0.187541,0.307309,0.222058,0.281878,0.215367,0.206606,0.249248,0.314304,0.506424,0.351874
4,HOBBIES_1_003_CA_1_validation,0.553571,0.573497,0.573497,0.853641,0.990462,1.214733,0.752704,0.643707,0.703883,...,0.822073,1.291438,0.853358,0.579836,0.385890,0.399994,0.424381,0.315821,0.458312,0.282702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_825_WI_3_evaluation,0.729797,0.506664,0.506193,0.628284,0.628284,0.864995,0.896318,0.931463,0.810668,...,0.669074,1.182386,1.232796,0.819411,1.181010,1.258080,0.672304,0.715482,0.802713,0.787410
60976,FOODS_3_826_WI_3_validation,1.141966,1.141966,0.961362,1.049447,0.916851,1.414627,1.642313,1.391382,1.337184,...,0.267267,0.678971,0.712857,0.517082,0.683976,0.549995,0.560926,0.638768,0.875055,1.274806
60977,FOODS_3_826_WI_3_evaluation,0.800908,0.836843,0.600648,0.403938,0.359735,0.463463,0.458224,0.312528,0.319353,...,0.268924,0.320200,0.410453,0.459328,0.917453,0.707557,0.631935,0.609926,0.848928,0.779769
60978,FOODS_3_827_WI_3_validation,1.366444,1.513490,1.803238,1.959945,1.643067,1.397115,1.499920,1.120503,1.226091,...,1.300980,1.733566,1.524560,1.355032,1.858123,2.028590,2.004649,1.769121,2.109157,1.843197
